In [2]:
import librosa.display
import librosa.util
import matplotlib.pyplot as plt
from IPython.display import Audio
import os
import scipy
from scipy.io import wavfile
import numpy as np
import utils.synthesis_util as su
import utils.model_util as mu
from utils.metrics import Metrics
from pesq_lib.pypesq import pypesq

## Examine dataset

In [3]:
audio_files = su.load_files()

### Clean audio

In [4]:
y_clean,sr = su.get_audio(audio_id=69,audio_files=audio_files)
print(y_clean.shape)
Audio(y_clean,rate=sr)

(21042,)


## Noisy audio

In [5]:
y_noise, sr = su.get_noisy_audio(audio_id=69,audio_files=audio_files)
print(y_noise.shape)
Audio(y_noise,rate=sr)

(21042,)


In [6]:
win = scipy.signal.hamming(256,"periodic")
magnitude,phase = su.audio_to_sttft(y_noise,win)

In [21]:
# 

In [7]:
predictors = su.get_predictors(magnitude)

In [8]:
predictors = np.array(predictors)
predictors.shape

(324, 129, 8)

## Load module

In [9]:
model = mu.load_model(n_features=129,n_segments=8,model_to_test="Baseline_FullyConnected/0505_130215")

## Denoise

In [10]:
length = len(y_noise)

In [11]:
sample = su.obtain_sample(predictors)

In [12]:
# print(model)
audio_rec = su.denoise_audio(model,sample,phase,win,length)
print(audio_rec.shape)
Audio(audio_rec,rate=sr)

(21042,)


# Calculate metrics

In [13]:
magnitude_clean,phase_clean = su.audio_to_sttft(y_clean,win)
magnitude_noise,phase_noise = su.audio_to_sttft(y_noise,win)
magnitude_denoised,phase_denoised = su.audio_to_sttft(audio_rec,win)
print(f"SNR_db for clean: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_clean, sr)))}")
print(f"SNR_db for noise: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_noise, sr)))}")
print(f"SNR_db for denoised: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_denoised, sr)))}")


SNR_db for clean: 30.01601325811101
SNR_db for noise: 28.151807644626253
SNR_db for denoised: 31.33245839657482


In [14]:
print(sr)

8000.0


In [15]:
print(f"PESQ value with narrowband (nb) considering original signal and original+noise: {pypesq(sr, y_clean, y_noise, 'nb')}")
print(f"PESQ value with narrowband (nb) considering original signal and reconstructed signal: {pypesq(sr, y_clean, audio_rec, 'nb')}")

PESQ value with narrowband (nb) considering original signal and original+noise: 1.609538197517395
PESQ value with narrowband (nb) considering original signal and reconstructed signal: 1.3559975624084473
